# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone


## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis: Results](#analysis)
* [Conclusion](#conclusion)



## Introduction: Business problem <a name="introduction"></a>

In this project I will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **Italian restaurant** in **New York City**, United States. Since there are lots of restaurants in Manhattan we will try to detect **locations that are not already crowded with restaurants**. We are also particularly interested in **areas with no Italian restaurants in vicinity**. I will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to Italian restaurants in the neighborhood, if any
* distance of neighborhood from Manhattan

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Manhattan will be obtained using **Google Maps API geocoding** of NYC location.

### Neighborhood Candidates

1. Download and Explore Dataset

Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 


In [1]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


#### Load and explore the data

Next, let's load data and take a look at the data

In [2]:
import numpy as np

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [3]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

Notice how all the relevant data is in the features key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.
And then let's take a look at the first item in this list.

In [4]:
neighborhoods_data = newyork_data['features']

In [5]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

Looking good. Let's now place all this into a Pandas dataframe.

#### Tranform the data into a _pandas_ dataframe

The next task is essentially transforming this data of nested Python dictionaries into a pandas dataframe. So let's start by creating an empty dataframe.

Foursquare credentials are defined in hidden cell bellow.

In [6]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.And then let's loop through the data and fill the dataframe one row at a time.

In [7]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [8]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.

In [9]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas of New York City that have low restaurant density, particularly those with low number of Italian restaurants. We will limit our analysis to Manhattan area.

In first step we have collected the required **data: location and type (category) of every restaurant within 1 km from Washington Heights**. We have also **identified Italian restaurants** (according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of '**restaurant density**' across different areas of Manhattan - we will use **folium** to identify a few promising areas close to each neighborhood with low number of restaurants in general (*and* no Italian restaurants in vicinity) and focus our attention on those areas.

In third and final step we will focus on most promising areas and within those create **clusters of locations that meet some basic requirements** established in discussion with stakeholders: we will take into consideration locations with **no more than three restaurants in radius of 1000 meters**. We will present map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final the categorized neighborhood in Manhattan area and search for optimal venue location by stakeholders.

#### Use geopy library to get the latitide and longitude values of New York City

In order to define an instance ot the geocoder, we need to define a user_agent.We will name our agent ny_explorer, as shown below.

In [11]:
from geopy.geocoders import Nominatim

In [12]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [13]:
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

In [14]:
!pip install folium

     |████████████████████████████████| 93 kB 3.4 MB/s  eta 0:00:01


In [15]:
import folium

Create a map of New York with Neighborhood superimposed on top

In [16]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lon, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7, parse_html=False).add_to(map_newyork)

map_newyork

For illustration purpose, let's simplify the above map and segment and cluster only the neighborhood in Manhattan. 
Let's slice the original dataframe and create a new dataframe of the Manhattab data.

In [17]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


Let's get the geographical coordinates of Manhattan.

In [18]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Manhattan are 40.7896239, -73.9598939.


As we did with all of New York City. Let's visualize Manhattan the neighborhoods in it.

In [19]:
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lon, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7, parse_html=False).add_to(map_manhattan)

map_manhattan

#### Define Foursquare Credentials and Version

Next, I am going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [20]:
CLIENT_ID = '4EOFWRD5RWV5HAERXE3I0BACHO2O0YZJROL5UVZ5LRTOTMEU'
CLIENT_SECRET = '32MVNIXJMA2DMGBGJI2YZ32TOUAELKV0L5QF4EJ4WMIT5GEN'
VERSION = '20210102'
LIMIT = 100

print('Your Credentials:')
print('CLIENT_ID:'+ CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your Credentials:
CLIENT_ID:4EOFWRD5RWV5HAERXE3I0BACHO2O0YZJROL5UVZ5LRTOTMEU
CLIENT_SECRET:32MVNIXJMA2DMGBGJI2YZ32TOUAELKV0L5QF4EJ4WMIT5GEN


Let's explore the first neighborhood in our dataframe.

In [21]:
manhattan_data.loc[2, 'Neighborhood']

'Washington Heights'

In [22]:
neighborhood_latitude = manhattan_data.loc[2, 'Latitude']
neighborhood_longitude = manhattan_data.loc[2, 'Longitude']

neighborhood_name = manhattan_data.loc[2, 'Neighborhood']
print('Latitude and Longitude values of {}, {}.'.format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Latitude and Longitude values of Washington Heights, 40.85190252555305.


Now, let's get the top 100 value that are in Washington Height with a radius of 1000 meters.
And then let's create the GET request URL. Name my URL url.

In [23]:
LIMIT = 100
radius = 1000


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=4EOFWRD5RWV5HAERXE3I0BACHO2O0YZJROL5UVZ5LRTOTMEU&client_secret=32MVNIXJMA2DMGBGJI2YZ32TOUAELKV0L5QF4EJ4WMIT5GEN&v=20210102&ll=40.85190252555305,-73.93690027985234&radius=1000&limit=100'

From the Foursquare lab in the previous module, we know that all the information is in the _item_ key. Before we proceed. Let's borrow the get_category_type function from the Foursquare lab.

In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ff1448d5246d6620b6bd992'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Hudson Heights',
  'headerFullLocation': 'Hudson Heights, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 147,
  'suggestedBounds': {'ne': {'lat': 40.86090253455306,
    'lng': -73.92502406180459},
   'sw': {'lat': 40.84290251655304, 'lng': -73.94877649790008}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '59f76cd528374e40d82f761b',
       'name': 'The Uptown Garrison',
       'location': {'address': '821 West 181st street',
        'crossStreet': 'Pinehurst',
        'lat': 40.85125539385438,
       

In [25]:
def get_category_type(row):
    try:
        cagegories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list) == 2:
        return None
    else: 
        return categories_list[2]['name']

Now we are ready to clean the json and structure into a _pandas_ dataframe.

Let's create a function to repeat the same process to all the neighborhoods in Manhattan.

In [43]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [44]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


Now write the code to run the above function on each neighborhood and create a new dataframe called _manhattan_venues_.

In [45]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [46]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3978, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
1,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Sam's Pizza,40.879435,-73.905859,Pizza Place
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


After checking the size of resulting dataframe (above) and then check how many vanues were returned for each Neighborhood.

In [47]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,100,100,100,100,100,100
Carnegie Hill,100,100,100,100,100,100
Central Harlem,100,100,100,100,100,100
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,100,100,100,100,100,100
East Village,100,100,100,100,100,100


Let's find out how many unique categories can be curated from all the returned venues.

In [48]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 315 uniques categories.


## Analysis: Results

Analyze Each Neighborhood

In [49]:
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Butcher,Cafeteria,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Arts Building,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Puerto Rican Restaurant,Ramen Restaurant,Recording Studio,Rental Car Location,Reservoir,Resort,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,State / Provincial Park,Steakhouse,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Synagogue,Szechuan R

In [50]:
manhattan_onehot.shape

(3978, 316)

Next, let's gruop rows by neighborhood and by taking the mean of the frequency of occurence of each category and then confirm the new size.

In [51]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Butcher,Cafeteria,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Arts Building,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Puerto Rican Restaurant,Ramen Restaurant,Recording Studio,Rental Car Location,Reservoir,Resort,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,State / Provincial Park,Steakhouse,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Synagogue,Szechuan R

In [52]:
manhattan_grouped.shape

(40, 316)

Let's print each neighborhood along with the top 5 most common venues.

In [53]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
         venue  freq
0  Coffee Shop  0.09
1         Park  0.07
2        Hotel  0.06
3        Plaza  0.03
4          Gym  0.03


----Carnegie Hill----
                  venue  freq
0           Coffee Shop  0.07
1           Yoga Studio  0.05
2                  Café  0.05
3  Gym / Fitness Center  0.04
4           Pizza Place  0.04


----Central Harlem----
                             venue  freq
0  Southern / Soul Food Restaurant  0.06
1                           Lounge  0.04
2               Seafood Restaurant  0.03
3                              Bar  0.03
4                French Restaurant  0.03


----Chelsea----
                 venue  freq
0          Art Gallery  0.10
1          Coffee Shop  0.06
2               Bakery  0.04
3   Seafood Restaurant  0.03
4  American Restaurant  0.03


----Chinatown----
            venue  freq
0    Optical Shop  0.05
1            Café  0.05
2  Ice Cream Shop  0.04
3        Wine Bar  0.04
4    Cocktail Bar  0.04


----Civic Cente

Let's put that into a _pandas_ datafram. 

First, let's write a function to sort the venues in descending order.

In [54]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [55]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Park,Hotel,Memorial Site,Gym,Plaza,Burger Joint,American Restaurant,Sandwich Place,Clothing Store
1,Carnegie Hill,Coffee Shop,Yoga Studio,Café,Gym / Fitness Center,Pizza Place,Gym,Bakery,Art Museum,Wine Shop,Bar
2,Central Harlem,Southern / Soul Food Restaurant,Lounge,Sushi Restaurant,Gym / Fitness Center,African Restaurant,Seafood Restaurant,Bar,Café,Pizza Place,French Restaurant
3,Chelsea,Art Gallery,Coffee Shop,Bakery,Seafood Restaurant,American Restaurant,New American Restaurant,Beer Bar,Bagel Shop,Salon / Barbershop,Roof Deck
4,Chinatown,Optical Shop,Café,Cocktail Bar,Wine Bar,Sandwich Place,Chinese Restaurant,Ice Cream Shop,Mexican Restaurant,Coffee Shop,Bakery


#### Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [56]:
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

kmeans.labels_[0:10] 

array([2, 0, 1, 4, 0, 0, 2, 3, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [57]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,3,Park,Pizza Place,Sandwich Place,Café,Spanish Restaurant,Donut Shop,Mexican Restaurant,Discount Store,Pharmacy,Scenic Lookout
1,Manhattan,Chinatown,40.715618,-73.994279,0,Optical Shop,Café,Cocktail Bar,Wine Bar,Sandwich Place,Chinese Restaurant,Ice Cream Shop,Mexican Restaurant,Coffee Shop,Bakery
2,Manhattan,Washington Heights,40.851903,-73.936900,3,Pizza Place,Café,Bakery,Latin American Restaurant,Tapas Restaurant,Coffee Shop,Park,Grocery Store,Bar,Spanish Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,3,Latin American Restaurant,Deli / Bodega,Mexican Restaurant,Restaurant,Café,Wine Bar,Spanish Restaurant,Pizza Place,Bakery,Park
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1,Coffee Shop,Bar,Deli / Bodega,Mexican Restaurant,Park,Café,Ethiopian Restaurant,Sushi Restaurant,Yoga Studio,American Restaurant


Finally, let's visualize the resulting clusters.

In [58]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examime Clusters

Now, I can examine each cluster and determine venue categories that dustinguish each cluster. Based on the defining categories, I can assign a name to each cluster.

##### Cluster 1

In [59]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Optical Shop,Café,Cocktail Bar,Wine Bar,Sandwich Place,Chinese Restaurant,Ice Cream Shop,Mexican Restaurant,Coffee Shop,Bakery
8,Upper East Side,Exhibit,Italian Restaurant,Bakery,Coffee Shop,Gym / Fitness Center,American Restaurant,Hotel,Yoga Studio,Bar,Outdoor Sculpture
10,Lenox Hill,Italian Restaurant,Café,Gym / Fitness Center,Sushi Restaurant,Cocktail Bar,Coffee Shop,Burger Joint,French Restaurant,Gym,Dessert Shop
13,Lincoln Square,Italian Restaurant,French Restaurant,Jazz Club,Performing Arts Venue,Plaza,Theater,Coffee Shop,Gym / Fitness Center,Café,Indie Movie Theater
18,Greenwich Village,Italian Restaurant,Sandwich Place,Café,Seafood Restaurant,Sushi Restaurant,Coffee Shop,Gym,American Restaurant,Pizza Place,Comedy Club
20,Lower East Side,Wine Shop,Café,Wine Bar,Bakery,Vietnamese Restaurant,Italian Restaurant,Coffee Shop,Asian Restaurant,Grocery Store,Pizza Place
22,Little Italy,Optical Shop,Italian Restaurant,Sandwich Place,Cosmetics Shop,Hotel,Café,Paper / Office Supplies Store,Dessert Shop,Women's Store,Jewelry Store
23,Soho,Italian Restaurant,Bakery,Café,Cosmetics Shop,Mediterranean Restaurant,Sandwich Place,Clothing Store,French Restaurant,Optical Shop,Coffee Shop
30,Carnegie Hill,Coffee Shop,Yoga Studio,Café,Gym / Fitness Center,Pizza Place,Gym,Bakery,Art Museum,Wine Shop,Bar
31,Noho,Cocktail Bar,Vietnamese Restaurant,Coffee Shop,Italian Restaurant,Japanese Restaurant,Hotel,Wine Bar,Pizza Place,Rock Club,Asian Restaurant


##### Cluster 2

In [60]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Hamilton Heights,Coffee Shop,Bar,Deli / Bodega,Mexican Restaurant,Park,Café,Ethiopian Restaurant,Sushi Restaurant,Yoga Studio,American Restaurant
5,Manhattanville,Park,Coffee Shop,American Restaurant,Italian Restaurant,Mexican Restaurant,Seafood Restaurant,Café,Cocktail Bar,Historic Site,Indian Restaurant
6,Central Harlem,Southern / Soul Food Restaurant,Lounge,Sushi Restaurant,Gym / Fitness Center,African Restaurant,Seafood Restaurant,Bar,Café,Pizza Place,French Restaurant
9,Yorkville,Ice Cream Shop,Coffee Shop,Italian Restaurant,Deli / Bodega,Gym,Yoga Studio,Bakery,Japanese Restaurant,Pizza Place,Bagel Shop
12,Upper West Side,Italian Restaurant,Gym,Bar,Café,Park,Bakery,Coffee Shop,Yoga Studio,Wine Bar,American Restaurant
19,East Village,Wine Bar,Bar,Pizza Place,Vegetarian / Vegan Restaurant,Coffee Shop,Cocktail Bar,Dessert Shop,Korean Restaurant,Vietnamese Restaurant,Bagel Shop
25,Manhattan Valley,Coffee Shop,Grocery Store,Indian Restaurant,Wine Shop,Park,Mexican Restaurant,Pizza Place,Playground,Bar,Ice Cream Shop
26,Morningside Heights,Coffee Shop,Park,American Restaurant,Chinese Restaurant,Italian Restaurant,Seafood Restaurant,Bakery,Bookstore,Grocery Store,Playground
29,Financial District,Coffee Shop,Pizza Place,Hotel,Café,Gym,Cocktail Bar,Memorial Site,Juice Bar,Steakhouse,Mediterranean Restaurant
37,Stuyvesant Town,Bar,Pizza Place,Park,Cocktail Bar,Coffee Shop,Bagel Shop,American Restaurant,Gym / Fitness Center,Gourmet Shop,Filipino Restaurant


##### Cluster 3

In [61]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Roosevelt Island,Park,Sushi Restaurant,Coffee Shop,Café,Greek Restaurant,Deli / Bodega,Tennis Court,Pizza Place,Italian Restaurant,Cocktail Bar
14,Clinton,Hotel,Coffee Shop,Theater,Gym / Fitness Center,Bar,American Restaurant,Burger Joint,Gym,Bakery,Italian Restaurant
15,Midtown,Theater,Plaza,Hotel,Gym,Coffee Shop,Italian Restaurant,Gourmet Shop,Bookstore,Food Truck,Sushi Restaurant
16,Murray Hill,Korean Restaurant,Gym / Fitness Center,Coffee Shop,Hotel,Grocery Store,Bakery,Gourmet Shop,Pizza Place,Building,Boxing Gym
21,Tribeca,Park,Spa,Coffee Shop,Hotel,Wine Bar,Greek Restaurant,Gym / Fitness Center,French Restaurant,American Restaurant,Italian Restaurant
24,West Village,Park,American Restaurant,Italian Restaurant,Coffee Shop,Wine Bar,French Restaurant,Cocktail Bar,Jazz Club,Steakhouse,Pizza Place
27,Gramercy,Park,American Restaurant,Italian Restaurant,Juice Bar,Coffee Shop,Mexican Restaurant,New American Restaurant,Indian Restaurant,Furniture / Home Store,Bar
28,Battery Park City,Coffee Shop,Park,Hotel,Memorial Site,Gym,Plaza,Burger Joint,American Restaurant,Sandwich Place,Clothing Store
33,Midtown South,Korean Restaurant,Coffee Shop,Gym / Fitness Center,Hotel,Italian Restaurant,Dessert Shop,Café,Japanese Restaurant,Yoga Studio,Boxing Gym
34,Sutton Place,Gym,Italian Restaurant,Hotel,Park,Bakery,Pizza Place,Vegetarian / Vegan Restaurant,Gym / Fitness Center,Spa,Salon / Barbershop


##### Cluster 4

In [62]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Park,Pizza Place,Sandwich Place,Café,Spanish Restaurant,Donut Shop,Mexican Restaurant,Discount Store,Pharmacy,Scenic Lookout
2,Washington Heights,Pizza Place,Café,Bakery,Latin American Restaurant,Tapas Restaurant,Coffee Shop,Park,Grocery Store,Bar,Spanish Restaurant
3,Inwood,Latin American Restaurant,Deli / Bodega,Mexican Restaurant,Restaurant,Café,Wine Bar,Spanish Restaurant,Pizza Place,Bakery,Park
7,East Harlem,Café,Mexican Restaurant,Pizza Place,Bakery,Deli / Bodega,Thai Restaurant,Italian Restaurant,Fountain,Playground,Gym


##### Cluster 5

In [63]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Chelsea,Art Gallery,Coffee Shop,Bakery,Seafood Restaurant,American Restaurant,New American Restaurant,Beer Bar,Bagel Shop,Salon / Barbershop,Roof Deck


## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify the low number of restaurants (particularly Italian restaurants) close to Washington Heights area in order to aid stakeholders in narrowing down the search for optimal location for a new Italian restaurant. By calculating restaurant density distribution from Foursquare data and considering the frequency of each categorized venue,I have first identified general boroughs that justify further analysis, and then generated extensive collection of locations which satisfy some basic requirements regarding existing nearby restaurants. Clustering of those locations was then performed in order to create major frequency of restaurant nearby and addresses of those neighborhoods were created to be used as starting points for final exploration by stakeholders.